In [1]:
#Add Graphframes library to databricks
from graphframes import *

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
# inputPath = "dbfs:/mnt/yelp-mount"

# businessDf = spark.read.json(inputPath+"/business.json")

In [3]:
business = sqlContext.read.json("/FileStore/tables/business.json")
business.registerTempTable("business")

In [4]:
%sql select * from business

In [5]:
userdf = sqlContext.read.json("/FileStore/tables/user.json")
userdf_exploded = userdf.withColumn('friends', explode('friends'))
userdf_exploded.registerTempTable("userTable")

In [6]:
%sql select * from userTable

In [7]:
userVertices = userdf_exploded.withColumnRenamed("user_id", "id").distinct()
friendsEdges = userdf_exploded.withColumnRenamed("user_id", "src").withColumnRenamed("friends", "dst")
  
  
Friendgraph = GraphFrame(userVertices,  friendsEdges)


In [8]:
display(userVertices)

In [9]:
display(Friendgraph.vertices.count())

In [10]:
display(Friendgraph.inDegrees)


In [11]:
results = Friendgraph.pageRank(resetProbability=0.15, tol=0.01)


In [12]:
display(results.vertices.select("id", "pagerank"))